# Reproducing table 6

This notebook currently imports results from a run of PHC.py and attempts to map them to Table 6.

Table 6 results are from Shoaib M, Ramamohan V. **Simulation modeling and analysis of primary health center operations**. *SIMULATION* 98(3):183-208. (2022). <https://doi.org/10.1177/00375497211030931>.

## Set up

In [1]:
# To run model
import PHC
import numpy as np

# To import and process results
import pandas as pd
import xlrd
import os

In [2]:
table6_path = '../original_study/tab6.csv'

## Run model

Set parameters for each configuration based on Table 3.

In [3]:
# For all: days 365, warm up 180, replication 10, 6 inpatient beds, 1 delivery
# bed, 3 staff nurse, 1 NCD nurse. These are default parameters in the model,
# and below, have just specified the parameters that changed between
# configurations

# TODO: Change all replication to 100 (just running with 10 for now for speed)

t6_c1_param = {
    'doc_cap': 2,
    'OPD_iat': 4,
    'IPD_iat': 2880,
    'delivery_iat': 1440,
    'ANC_iat': 1440,
    'rep_file': 't6_c1.xls',
}

t6_c2_param = {
    'doc_cap': 1,
    'OPD_iat': 9,
    'IPD_iat': 2880,
    'delivery_iat': 2880,
    'ANC_iat': 2880,
    'rep_file': 't6_c2.xls',
}

t6_c3_param = {
    'doc_cap': 1,
    'OPD_iat': 9,
    'IPD_iat': 2880,
    'any_delivery': False,
    'any_ANC': False,
    'rep_file': 't6_c3.xls'
}

t6_c4_param = {
    'doc_cap': 2,
    'OPD_iat': 3,
    'mean': 5,
    'sd': 1,
    'consult_boundary_1': 2,
    'consult_boundary_2': 2,
    'IPD_iat': 2880,
    'delivery_iat': 1440,
    'ANC_iat': 1440,
    'rep_file': 't6_c4.xls'
}

Run model for each configuration (not running every time as takes apx. 8 minutes).

In [4]:
# Input configuration parameters to main() (have to append with 's_', and
# add ** as inputting dict as parameters)
# for config in [t6_c1_param, t6_c2_param, t6_c3_param, t6_c4_param]:
#     print(f'''Running: {config['rep_file']}''')
#     PHC.main(**{f's_{k}': v for k, v in config.items()})

## Import and process replication results

In [5]:
# Make dictionary with labels from table 6, and corresponding names from model output
t6_labels = {
  'doc occ': 'Doctor utilisation',
  'NCD occ': 'NCD Nurse utilisation',
  'staff nurse occ': 'Staff nurse utilisation',
  'pharm occ': 'Pharmacist utilisation',
  'lab occ': 'Lab utilisation',
  'ipd bed occ': 'Inpatient bed utilisation',
  'del occ': 'Labour bed utilisation',  # "Del" stands for delivery
  'opd q len': 'Mean length of OPD queue (number of patients)',
  'OPD Q wt': 'OPD queue waiting time (minutes)',
  'pharmacy q len': 'Mean length of pharmacy queue (number of patients)',
  'Pharmacy Q wt': 'Pharmacy queue waiting time (minutes)',
  'lab q len': 'Mean length of Lab queue (number of patients)',
  'Lab Q wt': 'Lab queue waiting time (minutes)',
  'prop_del_referred': 'Fraction of childbirth cases referred'
}

In [6]:
# List of files to loop through
files = ['t6_c1', 't6_c2', 't6_c3', 't6_c4']

# Empty list to store results
result_list = []

for f in files:
    # Import .xls and convert to pandas dataframe
    book = xlrd.open_workbook(os.path.join('outputs', f'{f}.xls'))
    result = pd.read_excel(book, header=None, index_col=0)

    # Add proportion of childbirth cases referred
    result.loc['prop_del_referred'] = (
        result.loc['del referred'] / result.loc['Del patients'])

    # Find mean and standard deviation from the replication
    # Save as dataframe, dropping the duplicate rows (NCD occ twice)
    res = pd.DataFrame({
        f'model_{f}_mean': result.mean(axis=1),
        f'model_{f}_sd': result.std(axis=1)
    }).drop_duplicates()

    # Save to list
    result_list.append(res)

In [7]:
# Combine into single dataframe
summary = (pd.concat(result_list, axis=1)
           .reset_index()
           .rename(columns= {0: 'model_outcome'}))

# Add labels to model results
summary['t6_outcome'] = summary['model_outcome'].map(t6_labels)

summary

,model_outcome,model_t6_c1_mean,model_t6_c1_sd,model_t6_c2_mean,model_t6_c2_sd,model_t6_c3_mean,model_t6_c3_sd,model_t6_c4_mean,model_t6_c4_sd,t6_outcome
0,OPD patients,33133.600000,179.979135,14872.400000,1.386588e+02,14933.900000,53.875267,44161.600000,207.882018,NaN
1,IPD patients,184.000000,13.703203,190.300000,1.391282e+01,181.600000,13.309980,180.200000,11.773794,NaN
2,ANC patients,371.200000,12.839176,211.500000,1.314238e+01,0.000000,0.000000,373.900000,14.371654,NaN
3,Del patients,363.000000,25.064362,176.900000,8.252272e+00,NaN,NaN,363.100000,16.881614,NaN
4,OPD Q wt,0.009304,0.004588,0.180556,2.189756e-02,0.034672,0.000787,7.087377,0.171371,OPD queue waiting time (minutes)
5,Pharmacy Q wt,1.019412,0.012652,0.240693,4.652718e-03,0.231200,0.003927,1.277731,0.025166,Pharmacy queue waiting time (minutes)
6,Lab Q wt,2.087956,0.059170,0.605414,2.874245e-02,0.570976,0.014748,3.170211,0.061232,Lab queue waiting time (minutes)
7,doc occ,0.269412,0.003003,0.372110,2.163915e-03,0.355399,0.002388,1.145836,0.004580,Doctor utilisation
8,Lab patient list,189463.400000,104190.501449,84293.200000,4.652751e+04,86041.800000,47115.631487,254249.200000,139835.319294,NaN
9,OPD q len,0.008821,0.003825,0.167379,3.207569e-02,0.034586,0.000754,6.932062,0.304803,NaN


## Import table 6 results and compare against run results

In [8]:
# Import table 6
t6 = pd.read_csv(table6_path).rename(columns={'outcome': 't6_outcome'})
t6

,t6_outcome,config1_mean,config1_sd,config2_mean,config2_sd,config3_mean,config3_sd,benchmark_mean,benchmark_sd
0,Doctor utilisation,0.268,0.003,0.372,0.004,0.354,0.002,1.142,0.006
1,NCD Nurse utilisation,0.865,0.011,0.469,0.005,0.468,0.005,1.232,0.019
2,Staff nurse utilisation,0.323,0.008,0.243,0.006,0.160,0.001,0.322,0.008
3,Pharmacist utilisation,0.643,0.004,0.288,0.003,0.289,0.003,0.855,0.005
4,Lab utilisation,0.559,0.008,0.254,0.004,0.239,0.004,0.736,0.011
5,Inpatient bed utilisation,0.093,0.004,0.055,0.003,0.011,0.001,0.093,0.004
6,Labour bed utilisation,0.283,0.010,0.153,0.009,NaN,NaN,0.281,0.012
7,Mean length of OPD queue (number of patients),0.000,0.000,0.007,0.001,0.001,0.000,0.817,0.027
8,OPD queue waiting time (minutes),0.009,0.004,0.171,0.032,0.034,0.001,6.789,0.268
9,Mean length of pharmacy queue (number of patie...,0.090,0.002,0.010,0.001,0.009,0.000,0.150,0.002


In [9]:
# Merge
compare = t6.merge(summary)
compare.head()

,t6_outcome,config1_mean,config1_sd,config2_mean,config2_sd,config3_mean,config3_sd,benchmark_mean,benchmark_sd,model_outcome,model_t6_c1_mean,model_t6_c1_sd,model_t6_c2_mean,model_t6_c2_sd,model_t6_c3_mean,model_t6_c3_sd,model_t6_c4_mean,model_t6_c4_sd
0,Doctor utilisation,0.268,0.003,0.372,0.004,0.354,0.002,1.142,0.006,doc occ,0.269412,0.003003,0.372110,0.002164,0.355399,0.002388,1.145836,0.004580
1,NCD Nurse utilisation,0.865,0.011,0.469,0.005,0.468,0.005,1.232,0.019,NCD occ,0.869785,0.011039,0.469595,0.005111,0.470006,0.006064,1.231760,0.022432
2,Staff nurse utilisation,0.323,0.008,0.243,0.006,0.160,0.001,0.322,0.008,staff nurse occ,0.322609,0.008560,0.242049,0.004190,0.160632,0.001316,0.321931,0.006419
3,Pharmacist utilisation,0.643,0.004,0.288,0.003,0.289,0.003,0.855,0.005,pharm occ,0.642954,0.003603,0.288769,0.003379,0.289968,0.001035,0.856621,0.005442
4,Lab utilisation,0.559,0.008,0.254,0.004,0.239,0.004,0.736,0.011,lab occ,0.557508,0.004960,0.253362,0.006092,0.240857,0.003743,0.738856,0.012791


In [10]:
compare_col = [
    ('config1_mean', 'model_t6_c1_mean'),
    ('config1_sd', 'model_t6_c1_sd'),
    ('config2_mean', 'model_t6_c2_mean'),
    ('config2_sd', 'model_t6_c2_sd'),
    ('config3_mean', 'model_t6_c3_mean'),
    ('config3_sd', 'model_t6_c3_sd'),
    ('benchmark_mean', 'model_t6_c4_mean'),
    ('benchmark_sd', 'model_t6_c4_sd'),]

for col in compare_col:
    # Find difference between two columns
    compare[f'change_{col[1]}'] = abs(compare[col[1]] - compare[col[0]])
    # Find percent change between two columns
    subset = compare[list(col)]
    pct_change = subset.pct_change(axis=1).iloc[:, 1]*100
    compare[f'pct_change_{col[1]}'] = pct_change

# Display each of the results
for col in compare_col:
    # Set outcome as index, and get the two results plus percent change
    subset = compare.set_index('t6_outcome')[
        list(col) + [f'change_{col[1]}', f'pct_change_{col[1]}']]
    display(subset)

,config1_mean,model_t6_c1_mean,change_model_t6_c1_mean,pct_change_model_t6_c1_mean
t6_outcome,,,,
Doctor utilisation,0.268,0.269412,0.001412,0.526750
NCD Nurse utilisation,0.865,0.869785,0.004785,0.553143
Staff nurse utilisation,0.323,0.322609,0.000391,-0.121114
Pharmacist utilisation,0.643,0.642954,0.000046,-0.007225
Lab utilisation,0.559,0.557508,0.001492,-0.266928
Inpatient bed utilisation,0.093,0.093243,0.000243,0.261063
Labour bed utilisation,0.283,0.287000,0.004000,1.413428
Mean length of OPD queue (number of patients),0.000,0.000822,0.000822,inf
OPD queue waiting time (minutes),0.009,0.009304,0.000304,3.380654


,config1_sd,model_t6_c1_sd,change_model_t6_c1_sd,pct_change_model_t6_c1_sd
t6_outcome,,,,
Doctor utilisation,0.003,0.003003,0.000003,0.088112
NCD Nurse utilisation,0.011,0.011039,0.000039,0.355096
Staff nurse utilisation,0.008,0.008560,0.000560,6.997176
Pharmacist utilisation,0.004,0.003603,0.000397,-9.937006
Lab utilisation,0.008,0.004960,0.003040,-37.996691
Inpatient bed utilisation,0.004,0.005137,0.001137,28.424692
Labour bed utilisation,0.010,0.012517,0.002517,25.166556
Mean length of OPD queue (number of patients),0.000,0.000406,0.000406,inf
OPD queue waiting time (minutes),0.004,0.004588,0.000588,14.702129


,config2_mean,model_t6_c2_mean,change_model_t6_c2_mean,pct_change_model_t6_c2_mean
t6_outcome,,,,
Doctor utilisation,0.372,0.372110,0.000110,0.029651
NCD Nurse utilisation,0.469,0.469595,0.000595,0.126811
Staff nurse utilisation,0.243,0.242049,0.000951,-0.391160
Pharmacist utilisation,0.288,0.288769,0.000769,0.266990
Lab utilisation,0.254,0.253362,0.000638,-0.251295
Inpatient bed utilisation,0.055,0.055268,0.000268,0.487291
Labour bed utilisation,0.153,0.150000,0.003000,-1.960784
Mean length of OPD queue (number of patients),0.007,0.007131,0.000131,1.867144
OPD queue waiting time (minutes),0.171,0.180556,0.009556,5.588177


,config2_sd,model_t6_c2_sd,change_model_t6_c2_sd,pct_change_model_t6_c2_sd
t6_outcome,,,,
Doctor utilisation,0.004,0.002164,0.001836,-45.902124
NCD Nurse utilisation,0.005,0.005111,0.000111,2.221094
Staff nurse utilisation,0.006,0.004190,0.001810,-30.161748
Pharmacist utilisation,0.003,0.003379,0.000379,12.648057
Lab utilisation,0.004,0.006092,0.002092,52.305764
Inpatient bed utilisation,0.003,0.002650,0.000350,-11.650653
Labour bed utilisation,0.009,0.008165,0.000835,-9.278158
Mean length of OPD queue (number of patients),0.001,0.000846,0.000154,-15.422369
OPD queue waiting time (minutes),0.032,0.021898,0.010102,-31.570127


,config3_mean,model_t6_c3_mean,change_model_t6_c3_mean,pct_change_model_t6_c3_mean
t6_outcome,,,,
Doctor utilisation,0.354,0.355399,0.001399,0.395336
NCD Nurse utilisation,0.468,0.470006,0.002006,0.428706
Staff nurse utilisation,0.160,0.160632,0.000632,0.395143
Pharmacist utilisation,0.289,0.289968,0.000968,0.334861
Lab utilisation,0.239,0.240857,0.001857,0.776935
Inpatient bed utilisation,0.011,0.011464,0.000464,4.218362
Labour bed utilisation,NaN,NaN,NaN,NaN
Mean length of OPD queue (number of patients),0.001,0.001374,0.000374,37.354263
OPD queue waiting time (minutes),0.034,0.034672,0.000672,1.975607


,config3_sd,model_t6_c3_sd,change_model_t6_c3_sd,pct_change_model_t6_c3_sd
t6_outcome,,,,
Doctor utilisation,0.002,0.002388,0.000388,19.408639
NCD Nurse utilisation,0.005,0.006064,0.001064,21.273241
Staff nurse utilisation,0.001,0.001316,0.000316,31.621657
Pharmacist utilisation,0.003,0.001035,0.001965,-65.493446
Lab utilisation,0.004,0.003743,0.000257,-6.413813
Inpatient bed utilisation,0.001,0.000896,0.000104,-10.377371
Labour bed utilisation,NaN,NaN,NaN,NaN
Mean length of OPD queue (number of patients),0.000,0.000035,0.000035,inf
OPD queue waiting time (minutes),0.001,0.000787,0.000213,-21.307875


,benchmark_mean,model_t6_c4_mean,change_model_t6_c4_mean,pct_change_model_t6_c4_mean
t6_outcome,,,,
Doctor utilisation,1.142,1.145836,0.003836,0.335941
NCD Nurse utilisation,1.232,1.231760,0.000240,-0.019499
Staff nurse utilisation,0.322,0.321931,0.000069,-0.021522
Pharmacist utilisation,0.855,0.856621,0.001621,0.189635
Lab utilisation,0.736,0.738856,0.002856,0.388032
Inpatient bed utilisation,0.093,0.093002,0.000002,0.002285
Labour bed utilisation,0.281,0.278000,0.003000,-1.067616
Mean length of OPD queue (number of patients),0.817,0.832990,0.015990,1.957175
OPD queue waiting time (minutes),6.789,7.087377,0.298377,4.395003


,benchmark_sd,model_t6_c4_sd,change_model_t6_c4_sd,pct_change_model_t6_c4_sd
t6_outcome,,,,
Doctor utilisation,0.006,0.004580,0.001420,-23.658852
NCD Nurse utilisation,0.019,0.022432,0.003432,18.063774
Staff nurse utilisation,0.008,0.006419,0.001581,-19.759524
Pharmacist utilisation,0.005,0.005442,0.000442,8.838985
Lab utilisation,0.011,0.012791,0.001791,16.278967
Inpatient bed utilisation,0.004,0.004686,0.000686,17.152748
Labour bed utilisation,0.012,0.011353,0.000647,-5.392298
Mean length of OPD queue (number of patients),0.027,0.022424,0.004576,-16.949071
OPD queue waiting time (minutes),0.268,0.171371,0.096629,-36.055513


## Format model results like table 6, and save

In [11]:
compare_col = [
    ['Configuration 1', 'model_t6_c1_mean', 'model_t6_c1_sd'],
    ['Configuration 2', 'model_t6_c2_mean', 'model_t6_c2_sd'],
    ['Configuration 3', 'model_t6_c3_mean', 'model_t6_c3_sd'],
    ['Benchmark Case', 'model_t6_c4_mean', 'model_t6_c4_sd']]

# Combine each pair of columns into string as in Table 6
for col in compare_col:
    compare[col[0]] = (
        round(compare[col[1]], 3).astype(str) + ' (' +
        round(compare[col[2]], 3).astype(str) + ')')

# Set outcome name as index, and select those columns
formatted_model_res = (compare
                       .set_index('t6_outcome')
                       .rename_axis('Simulation Outcome')[
                           [item[0] for item in compare_col]])
formatted_model_res

,Configuration 1,Configuration 2,Configuration 3,Benchmark Case
Simulation Outcome,,,,
Doctor utilisation,0.269 (0.003),0.372 (0.002),0.355 (0.002),1.146 (0.005)
NCD Nurse utilisation,0.87 (0.011),0.47 (0.005),0.47 (0.006),1.232 (0.022)
Staff nurse utilisation,0.323 (0.009),0.242 (0.004),0.161 (0.001),0.322 (0.006)
Pharmacist utilisation,0.643 (0.004),0.289 (0.003),0.29 (0.001),0.857 (0.005)
Lab utilisation,0.558 (0.005),0.253 (0.006),0.241 (0.004),0.739 (0.013)
Inpatient bed utilisation,0.093 (0.005),0.055 (0.003),0.011 (0.001),0.093 (0.005)
Labour bed utilisation,0.287 (0.013),0.15 (0.008),nan (nan),0.278 (0.011)
Mean length of OPD queue (number of patients),0.001 (0.0),0.007 (0.001),0.001 (0.0),0.833 (0.022)
OPD queue waiting time (minutes),0.009 (0.005),0.181 (0.022),0.035 (0.001),7.087 (0.171)
